In [48]:
import pandas as pd


In [49]:
solar = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/capacity_factor/Solar/ninja_pv_country_GB_merra-2_corrected.csv')
wind = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/capacity_factor/Wind/ninja_wind_country_GB_current-merra-2_corrected.csv')
demand = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/electricity_demand/uk_all_year_demand.csv')
wind.head()

,Unnamed: 0,time,national,offshore,onshore
0,0,01/01/1980 00:00,0.2024,0.2858,0.1496
1,1,01/01/1980 01:00,0.2016,0.2827,0.1502
2,2,01/01/1980 02:00,0.2015,0.2852,0.1485
3,3,01/01/1980 03:00,0.2032,0.2920,0.1470
4,4,01/01/1980 04:00,0.2090,0.3039,0.1489


In [50]:
onshore = wind[['time','onshore']]
offshore = wind[['time','offshore']]
offshore.head()

,time,offshore
0,01/01/1980 00:00,0.2858
1,01/01/1980 01:00,0.2827
2,01/01/1980 02:00,0.2852
3,01/01/1980 03:00,0.2920
4,01/01/1980 04:00,0.3039


In [51]:
solar = solar.rename(columns={'pv':'value'})
offshore = offshore.rename(columns={'offshore':'value'})
onshore = onshore.rename(columns={'onshore':'value'})
demand = demand.rename(columns={'timestamp':'time','demand':'value'})
onshore.head()

,time,value
0,01/01/1980 00:00,0.1496
1,01/01/1980 01:00,0.1502
2,01/01/1980 02:00,0.1485
3,01/01/1980 03:00,0.1470
4,01/01/1980 04:00,0.1489


In [52]:
def to_datetime(dat, col_to_datetime):
    dat[col_to_datetime] = pd.to_datetime(dat[col_to_datetime])
    return dat


In [53]:
offshore = to_datetime(offshore, 'time')
onshore = to_datetime(onshore, 'time')
solar = to_datetime(solar, 'time')
demand = to_datetime(demand, 'time')

In [54]:
def get_hour(dat, datetime):
    hour_series = dat[datetime].dt.hour
    return hour_series
    


In [55]:
offshore['hour'] = get_hour(offshore, 'time')
onshore['hour'] = get_hour(onshore, 'time')
solar['hour'] = get_hour(solar, 'time')
demand['hour'] = get_hour(demand, 'time')
demand.head()

,time,value,hour
0,2011-05-27 16:00:00,38649.090909,16
1,2011-05-27 17:00:00,37497.583333,17
2,2011-05-27 18:00:00,36146.333333,18
3,2011-05-27 19:00:00,34741.166667,19
4,2011-05-27 20:00:00,34722.666667,20


In [10]:
def get_month(dat, datetime):
    month_series = dat[datetime].dt.month
    return month_series

In [56]:
offshore['month'] = get_month(offshore, 'time')
onshore['month'] = get_month(onshore, 'time')
solar['month'] = get_month(solar, 'time')
demand['month'] = get_month(demand, 'time')

demand.head()

,time,value,hour,month
0,2011-05-27 16:00:00,38649.090909,16,5
1,2011-05-27 17:00:00,37497.583333,17,5
2,2011-05-27 18:00:00,36146.333333,18,5
3,2011-05-27 19:00:00,34741.166667,19,5
4,2011-05-27 20:00:00,34722.666667,20,5


In [57]:
def get_dayofweek(dat, datetime):
    dayofweek_series = dat[datetime].dt.dayofweek
    return dayofweek_series

In [58]:
offshore['dayofweek'] = get_dayofweek(offshore, 'time')
onshore['dayofweek'] = get_dayofweek(onshore, 'time')
solar['dayofweek'] = get_dayofweek(solar, 'time')
demand['dayofweek'] = get_dayofweek(demand, 'time')

solar.head()

,time,value,hour,month,dayofweek
0,1985-01-01 00:00:00,0.0,0,1,1
1,1985-01-01 01:00:00,0.0,1,1,1
2,1985-01-01 02:00:00,0.0,2,1,1
3,1985-01-01 03:00:00,0.0,3,1,1
4,1985-01-01 04:00:00,0.0,4,1,1


In [14]:
def get_dayofmonth(dat, datetime):
    dayofmonth_series = dat[datetime].dt.day
    return dayofmonth_series

In [59]:
offshore['dayofmonth'] = get_dayofmonth(offshore, 'time')
onshore['dayofmonth'] = get_dayofmonth(onshore, 'time')
solar['dayofmonth'] = get_dayofmonth(solar, 'time')
demand['dayofmonth'] = get_dayofmonth(demand, 'time')

solar.head()

,time,value,hour,month,dayofweek,dayofmonth
0,1985-01-01 00:00:00,0.0,0,1,1,1
1,1985-01-01 01:00:00,0.0,1,1,1,1
2,1985-01-01 02:00:00,0.0,2,1,1,1
3,1985-01-01 03:00:00,0.0,3,1,1,1
4,1985-01-01 04:00:00,0.0,4,1,1,1


In [60]:
def get_lags(dat, value):
    previous_three_hours = [1,2,3,]
    previous_day = [24,25,26,27]
    previous_week = [168,169,170,171]
    lags_required = previous_three_hours + previous_day + previous_week

    new_cols = dat.assign(**{f'value-{lag}': dat[value].shift(lag) for lag in lags_required})
    return new_cols

In [61]:
offshore = get_lags(offshore, 'value')
onshore = get_lags(onshore, 'value')
solar = get_lags(solar, 'value')
demand = get_lags(demand, 'value')
solar.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171
0,1985-01-01 00:00:00,0.0,0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-01-01 01:00:00,0.0,1,1,1,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-01-01 02:00:00,0.0,2,1,1,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-01-01 03:00:00,0.0,3,1,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-01-01 04:00:00,0.0,4,1,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
offshore = offshore.dropna()
onshore = onshore.dropna()
solar = solar.dropna()
demand = demand.dropna()
demand

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171
171,2011-06-03 19:00:00,33880.250000,19,6,4,3,35690.250000,36978.333333,38135.416667,35907.000000,37420.250000,38815.583333,39945.250000,34741.166667,36146.333333,37497.583333,38649.090909
172,2011-06-03 20:00:00,33181.000000,20,6,4,3,33880.250000,35690.250000,36978.333333,35405.833333,35907.000000,37420.250000,38815.583333,34722.666667,34741.166667,36146.333333,37497.583333
173,2011-06-03 21:00:00,33132.083333,21,6,4,3,33181.000000,33880.250000,35690.250000,35090.250000,35405.833333,35907.000000,37420.250000,33434.833333,34722.666667,34741.166667,36146.333333
174,2011-06-03 22:00:00,30041.916667,22,6,4,3,33132.083333,33181.000000,33880.250000,31313.083333,35090.250000,35405.833333,35907.000000,29708.750000,33434.833333,34722.666667,34741.166667
175,2011-06-03 23:00:00,27883.666667,23,6,4,3,30041.916667,33132.083333,33181.000000,29467.250000,31313.083333,35090.250000,35405.833333,27410.000000,29708.750000,33434.833333,34722.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66502,2018-12-27 14:00:00,36784.750000,14,12,3,27,36682.000000,36814.166667,36714.583333,32605.500000,32773.250000,32548.916667,31786.916667,39188.750000,38287.500000,38239.750000,38465.000000
66503,2018-12-27 15:00:00,37835.750000,15,12,3,27,36784.750000,36682.000000,36814.166667,33133.333333,32605.500000,32773.250000,32548.916667,40193.583333,39188.750000,38287.500000,38239.750000
66504,2018-12-27 16:00:00,40393.583333,16,12,3,27,37835.750000,36784.750000,36682.000000,35031.833333,33133.333333,32605.500000,32773.250000,42569.583333,40193.583333,39188.750000,38287.500000
66505,2018-12-27 17:00:00,42150.583333,17,12,3,27,40393.583333,37835.750000,36784.750000,36038.833333,35031.833333,33133.333333,32605.500000,44252.916667,42569.583333,40193.583333,39188.750000


In [63]:
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

class EnglandAndWalesHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Years Day', month=1, day=1, observance=next_monday),
        GoodFriday,
        EasterMonday,
        Holiday('Early May bank holiday',
                month=5, day=1, offset=DateOffset(weekday=MO(1))),
        Holiday('Spring bank holiday',
                month=5, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Summer bank holiday',
                month=8, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Christmas Day', month=12, day=25, observance=next_monday),
        Holiday('Boxing Day',
                month=12, day=26, observance=next_monday_or_tuesday)
    ]

def check_if_holiday(dat, datetime):
    cal = EnglandAndWalesHolidayCalendar()
    
    holidays = cal.holidays(start=dat[datetime].min(), end=dat[datetime].max())

    dat['holiday'] = dat['time'].dt.date.astype('datetime64').isin(holidays)
    return dat

In [64]:
solar = check_if_holiday(solar, 'time')
offshore = check_if_holiday(offshore, 'time')
onshore = check_if_holiday(onshore, 'time')
demand = check_if_holiday(demand, 'time')

In [66]:
onshore.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1980-08-01 03:00:00,0.2022,3,8,4,1,0.1860,0.1758,0.1756,0.1765,0.1722,0.1702,0.1805,0.1470,0.1485,0.1502,0.1496,False
172,1980-08-01 04:00:00,0.2165,4,8,4,1,0.2022,0.1860,0.1758,0.1778,0.1765,0.1722,0.1702,0.1489,0.1470,0.1485,0.1502,False
173,1980-08-01 05:00:00,0.2146,5,8,4,1,0.2165,0.2022,0.1860,0.1751,0.1778,0.1765,0.1722,0.1483,0.1489,0.1470,0.1485,False
174,1980-08-01 06:00:00,0.2028,6,8,4,1,0.2146,0.2165,0.2022,0.1683,0.1751,0.1778,0.1765,0.1475,0.1483,0.1489,0.1470,False
175,1980-08-01 07:00:00,0.1967,7,8,4,1,0.2028,0.2146,0.2165,0.1654,0.1683,0.1751,0.1778,0.1486,0.1475,0.1483,0.1489,False


In [22]:
offshore.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1980-08-01 03:00:00,0.2285,3,8,4,1,0.2170,0.2067,0.2095,0.2822,0.3081,0.3314,0.3443,0.2920,0.2852,0.2827,0.2858,False
172,1980-08-01 04:00:00,0.2347,4,8,4,1,0.2285,0.2170,0.2067,0.2617,0.2822,0.3081,0.3314,0.3039,0.2920,0.2852,0.2827,False
173,1980-08-01 05:00:00,0.2318,5,8,4,1,0.2347,0.2285,0.2170,0.2441,0.2617,0.2822,0.3081,0.3078,0.3039,0.2920,0.2852,False
174,1980-08-01 06:00:00,0.2265,6,8,4,1,0.2318,0.2347,0.2285,0.2338,0.2441,0.2617,0.2822,0.3119,0.3078,0.3039,0.2920,False
175,1980-08-01 07:00:00,0.2206,7,8,4,1,0.2265,0.2318,0.2347,0.2294,0.2338,0.2441,0.2617,0.3148,0.3119,0.3078,0.3039,False


In [23]:
solar.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1985-08-01 03:00:00,0.0,3,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
172,1985-08-01 04:00:00,0.0,4,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
173,1985-08-01 05:00:00,0.0,5,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
174,1985-08-01 06:00:00,0.0,6,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
175,1985-08-01 07:00:00,0.0,7,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [67]:
demand.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,2011-06-03 19:00:00,33880.250000,19,6,4,3,35690.250000,36978.333333,38135.416667,35907.000000,37420.250000,38815.583333,39945.250000,34741.166667,36146.333333,37497.583333,38649.090909,False
172,2011-06-03 20:00:00,33181.000000,20,6,4,3,33880.250000,35690.250000,36978.333333,35405.833333,35907.000000,37420.250000,38815.583333,34722.666667,34741.166667,36146.333333,37497.583333,False
173,2011-06-03 21:00:00,33132.083333,21,6,4,3,33181.000000,33880.250000,35690.250000,35090.250000,35405.833333,35907.000000,37420.250000,33434.833333,34722.666667,34741.166667,36146.333333,False
174,2011-06-03 22:00:00,30041.916667,22,6,4,3,33132.083333,33181.000000,33880.250000,31313.083333,35090.250000,35405.833333,35907.000000,29708.750000,33434.833333,34722.666667,34741.166667,False
175,2011-06-03 23:00:00,27883.666667,23,6,4,3,30041.916667,33132.083333,33181.000000,29467.250000,31313.083333,35090.250000,35405.833333,27410.000000,29708.750000,33434.833333,34722.666667,False


In [ ]:
demand.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/demand.csv')
solar.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/solar.csv')
offshore.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/offshore.csv')
onshore.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/onshore.csv')